In [ ]:
#@title Activation Patching

from transformer_lens import patching

# TODO
current_answer_tokens = answer_tokens[0]

# cache for one input and its pair
clean_tokens = model.to_tokens(prompts[0], prepend_bos=True)
corrupted_tokens = model.to_tokens(prompts[1], prepend_bos=True)
clean_tokens = clean_tokens.to(device)
corrupted_tokens = corrupted_tokens.to(device)
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)
clean_logit_diff = logits_to_ave_logit_diff(clean_logits, answer_tokens)
corrupted_logit_diff = logits_to_ave_logit_diff(corrupted_logits, answer_tokens)
print(f"Clean logit diff: {clean_logit_diff:.4f}")
print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")

def ioi_metric(
    logits: Float[Tensor, "batch seq d_vocab"], 
    answer_tokens: Float[Tensor, "batch 2"] = current_answer_tokens,
    corrupted_logit_diff: float = corrupted_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
) -> Float[Tensor, ""]:
    '''
    Linear function of logit diff, calibrated so that it equals 0 when performance is 
    same as on corrupted input, and 1 when performance is same as on clean input.
    '''
    patched_logit_diff = logits_to_ave_logit_diff(logits, answer_tokens)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)
                                                          

act_patch_resid_pre = patching.get_act_patch_resid_pre(
    model = model,
    corrupted_tokens = corrupted_tokens,
    clean_cache = clean_cache,
    patching_metric = ioi_metric
)

labels = [f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))]

imshow(
    act_patch_resid_pre, 
    labels={"x": "Position", "y": "Layer"},
    x=labels,
    title="resid_pre Activation Patching",
    width=600
)

In [ ]:
# Final word translations

from deep_translator import GoogleTranslator

translator = GoogleTranslator(src='fr', target='en')

final_word_translations = []
for item in french_data:
    final_word = item.split()[-1]
    translation = translator.translate(final_word)
    final_word_translations.append(translation)

write_data(final_word_translations, "final_word_translations.json")